In [373]:
from pyvis.network import Network
from itertools import combinations

## Make visualization

In [374]:
DIGITS = 3
plist = [n for n in primes(10**(DIGITS)-1) if n >= 10**(DIGITS-1)]
net = Network('600px','900px')
net.add_nodes(plist, size=[8]*len(plist))

for (p1,p2) in combinations(plist,2):
    if isnear(p1,p2):
        net.add_edge(p1,p2)

# get adjacency matrix
graph = net.get_adj_list()
        
# MAKE VISUALIZATION
net.toggle_physics(status=True)
net.prep_notebook()
net.show('netplot.html')

---
## Use csr representation

In [375]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import shortest_path, connected_components
from itertools import combinations
import numpy as np

In [376]:
def primes(n):
    """ Returns  a list of primes < n """
    sieve = [True] * n
    for i in range(3,int(n**0.5)+1,2):
        if sieve[i]:
            sieve[i*i::2*i]=[False]*((n-i*i-1)//(2*i)+1)
    return [2] + [i for i in range(3,n,2) if sieve[i]]

In [377]:
def isnear(p1,p2):
    """ Returns TRUE if p1 and p2 differ by one digit """
    string1 = str(p1)
    string2 = str(p2)
    count_diffs = 0
    for a, b in zip(string1, string2):
        if a!=b:
            if count_diffs: return False
            count_diffs += 1
    return count_diffs == 1

In [378]:
def isnear_str(string1,string2):
    """ Returns TRUE if p1 and p2 differ by one digit """
    count_diffs = 0
    for a, b in zip(string1, string2):
        if a!=b:
            if count_diffs: return False
            count_diffs += 1
    return count_diffs == 1

In [379]:
def get_path(plist,Pr, i, j):
    path = [int(plist[j])]
    k = j
    while Pr[i, k] != -9999:
        path.append(int(plist[Pr[i, k]]))
        k = Pr[i, k]
    return path[::-1]

In [380]:
%%time

DIGITS = 3

plist = [str(n) for n in primes(10**(DIGITS)-1) if n >= 10**(DIGITS-1)]

data = []
row_ind = []
col_ind = []

for (i,j) in combinations(range(len(plist)),2):
    if isnear_str(plist[i],plist[j]):
        data.append(1)
        row_ind.append(i)
        col_ind.append(j)

# build matrix sparse adjacency matrix
mat = csr_matrix((data,(row_ind,col_ind)),(len(plist),len(plist)))

Wall time: 5.98 ms


In [381]:
%%time

dist_matrix, pred = shortest_path(csgraph=mat, directed=False, return_predecessors=True)

# deal with the possibility that the graph has multiple connected components
if connected_components(mat, directed=False, return_labels=False) > 1:
    print('There are multiple connected components!')
    dist_matrix = np.where(dist_matrix == np.inf, -9999, dist_matrix)

# find longest path (graph diameter)
(i1,i2) = np.unravel_index(dist_matrix.argmax(), dist_matrix.shape)
pp = get_path(plist,pred,i1,i2)
print(len(pp)-1)
print(pp)

6
[389, 379, 179, 109, 101, 701, 761]
Wall time: 5.98 ms
